In [1]:
#| default_exp imgs_stats

# Statistics of images

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|export

import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import itertools as it

from flood_exercise import utils_func
from flood_exercise import const_vals as CONST


In [51]:
#|export

class ImgsStatistics():

  def __init__(self,
      path_to_imgs : str , # path to the folder that contains the images
      path_to_split_file : str , # path to the folder that contains the split data
      ):
      
      # get the path to the split files
      self.path_to_split_file = utils_func.load_list_paths(path_to_split_file,filter_file=False)

      # get the tiles paths
      self.list_of_files = utils_func.load_list_paths(path_to_imgs,filter_file = True)
      self.img_stats_df = self._iterate_tiles_()

      #get the split data 
      self.split_dfs = self._get_split_data_()

      # join between the split data and the images stats
      self.results = self.img_stats_df.merge(self.split_dfs, on=CONST.JOIN_COL_NAME,
                                        how = 'left').drop_duplicates(subset = CONST.PATH_STR)
      self.results.drop(CONST.DF_ID_COL_NAME,axis=1,inplace=True)

      self.count_regions = pd.DataFrame(self.results[CONST.REGION_STR].value_counts())
      self.count_split = pd.DataFrame(self.results[CONST.SPLIT_COL_NAME].value_counts())


  def _get_split_data_(self):

      self.collect_dfs = []
      for split_path in self.path_to_split_file:
         # get the split name
         split_type_str = split_path.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[1]

         # open the csv file and add column with the split group
         df = pd.read_csv(split_path,header=None)
         # to be caution - take the two columns and make one column with all the files names
         df =pd.DataFrame(sorted(it.chain(*df.values)))

         df.columns = [CONST.DF_ID_COL_NAME]

         df[CONST.SPLIT_COL_NAME] = split_type_str
         self.collect_dfs.append(df)

         split_dfs = pd.concat(self.collect_dfs)

         split_dfs[CONST.JOIN_COL_NAME]= split_dfs[CONST.DF_ID_COL_NAME].str.split(CONST.SPLIT_TILES_NAMES_STR2).str[:2].str.join(CONST.SPLIT_TILES_NAMES_STR2)


      return split_dfs


  def _get_region_name_(self,
                        tile_name : str , # path of image , assuming that the first word in the file name is the region name
                        ):
     """
     Takes a string of images from the type "region_id_label.tif and extract the region name.
     for example, for "Bolivia_23014_S2Hand.tif" it will return Bolivia
     Parameters:
     folder_path (str): The path to the folder containing TIFF files.

     Returns:
     string: Name of the region , extracted from the tile name 
     """
     region = tile_name.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[0]
     return region
  
  def _img_statistics_(self,
                       path : str , # path to image (tif file)
                       ):
     
     with rasterio.open(path) as src:
      
      src_arr = src.read()

      #convert 0 to nan , assuming 0 is no value and we don't want it to interrupt the staitistics
      src_arr  = np.where(src_arr==0, np.nan, src_arr )
      band_names = list(src.descriptions)

      collect_bands_stats = {}

      for band_name , index in zip(band_names,range(0,src.read().shape[0])):
         self.bands_stats_imgs = {CONST.STR_BAND_NAME : [] , CONST.STR_MEAN : [] , CONST.STR_STD : [] }
         arr = src_arr[index,:,:]

         #calculate mean
         mean = np.nanmean(arr)
         
         #calculate std
         std = np.nanstd(arr)

         collect_bands_stats[band_name +'_' + CONST.STR_MEAN] = mean
         collect_bands_stats[band_name +'_' + CONST.STR_STD] = std
         
   
     df_img_stats = pd.DataFrame.from_dict([collect_bands_stats])

     return df_img_stats


  def _iterate_tiles_(self):
   
   self.collect_info = {CONST.PATH_STR : [] , CONST.REGION_STR : []}
   self.collect_stats = []

   for path in tqdm(self.list_of_files):
      #get the region name using the get_region_name function
      region = self._get_region_name_(path)
      
      #collect the regn name and the path for the final table
      self.collect_info[CONST.PATH_STR].append(path)
      self.collect_info[CONST.REGION_STR].append(region)
         
      #get the image statistics
      df_img_stats = self._img_statistics_(path)
      self.collect_stats.append(df_img_stats)

   #organize datasets
   #organize the region info
   df1 = pd.DataFrame(self.collect_info)
   #ordanize the stats
   df2 = pd.concat(self.collect_stats)
   df2.reset_index(inplace=True)
   #cocatenate
   results = pd.concat([df1,df2],axis=1)
   
   #add names column so we can join between the split type and the image name 
   results[CONST.JOIN_COL_NAME] = results[CONST.PATH_STR].str.split(CONST.SPLIT_TILES_NAMES_STR1).str[-1].str.split(CONST.SPLIT_TILES_NAMES_STR2).str[:2].str.join(CONST.SPLIT_TILES_NAMES_STR2)

   return results



In [52]:
instance = ImgsStatistics(
  path_to_imgs = r'D:\git\flood_exercise\S2',
  path_to_split_file= r'D:\git\flood_exercise\split\flood_handlabeled' )

instance.results

 10%|▉         | 38/399 [00:02<00:27, 13.02it/s]

In [ ]:
instance.count_regions

,count
region,
USA,69
India,68
Ghana,53
Sri-Lanka,42
Paraguay,33
Mekong,30
Spain,30
Pakistan,28
Nigeria,18


In [ ]:
instance.count_split

,count
split,
train,227
valid,79
test,78
bolivia,15


In [ ]:
# instance.results.to_csv(r'D:\git\flood_exercise\RESULTS\imgs_stats.csv')
instance.count_split.to_csv(r'D:\git\flood_exercise\RESULTS\count_split.csv')
instance.count_regions.to_csv(r'D:\git\flood_exercise\RESULTS\count_regions.csv')